In [1]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [2]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [3]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [4]:
# # replace NAN's
# def rmNAN(df, col):
#     df.col = df.col.fillna(0)

In [5]:
sallyMetrics = loadMetricData('sallyKung', "metrics")
sallyMetrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-09 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN
1436,2016-05-09 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN
1437,2016-05-09 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN
1438,2016-05-09 23:58:00,NaN,NaN,NaN,NaN,NaN,NaN
1439,2016-05-09 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
sallyMetrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [7]:
# clean metric data

sallyMetrics.timestamp = pd.to_datetime(sallyMetrics.timestamp)

sallyMetrics.heartrate = sallyMetrics.heartrate.fillna(0)
sallyMetrics.steps = sallyMetrics.steps.fillna(0)
sallyMetrics.calories = sallyMetrics.calories.fillna(0)
sallyMetrics.gsr = sallyMetrics.gsr.fillna(0)
sallyMetrics.skintemp = sallyMetrics.skintemp.fillna(0)
sallyMetrics.airtemp = sallyMetrics.airtemp.fillna(0)

sallyMetrics.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
0,2016-05-02 00:00:00,0,4,1.709,0,0,0
1,2016-05-02 00:01:00,0,4,1.709,0,0,0
2,2016-05-02 00:02:00,0,4,1.709,0,0,0
3,2016-05-02 00:03:00,0,4,1.709,0,0,0
4,2016-05-02 00:04:00,0,4,1.709,0,0,0


In [8]:
sallyActivities = loadMetricData('sallyKung', "activities")
sallyActivities.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,type,actual seconds,steps,calories,minutes,heart rate avg,heart rate min,heart rate max,state,version,id
1,2016-05-09 13:55:47,2016-05-09T13:55:47Z,America/New_York,-240,2016-05-09 14:00:54,2016-05-09T14:00:54Z,America/New_York,-240,walk,307,587,23.500000,5.116667,110.824104,NaN,NaN,complete,1,walk__2016-05-09T135547
2,2016-05-09 14:20:24,2016-05-09T14:20:24Z,America/New_York,-240,2016-05-09 14:27:39,2016-05-09T14:27:39Z,America/New_York,-240,walk,415,792,43.400002,7.250000,114.530121,NaN,NaN,complete,1,walk__2016-05-09T142024
3,2016-05-09 19:39:27,2016-05-09T19:39:27Z,America/New_York,-240,2016-05-09 19:45:28,2016-05-09T19:45:28Z,America/New_York,-240,walk,361,750,46.299999,6.016667,121.648201,NaN,NaN,complete,1,walk__2016-05-09T193927
4,2016-05-09 20:30:38,2016-05-09T20:30:38Z,America/New_York,-240,2016-05-09 21:12:19,2016-05-09T21:12:19Z,America/New_York,-240,walk,2074,3714,190.500000,41.683333,110.398262,NaN,NaN,complete,1,walk__2016-05-09T203038
5,2016-05-09 21:59:16,2016-05-09T21:59:16Z,America/New_York,-240,2016-05-09 22:07:35,2016-05-09T22:07:35Z,America/New_York,-240,walk,258,499,31.400000,8.316667,119.693802,NaN,NaN,complete,1,walk__2016-05-09T215916


In [9]:
sallySleep = loadMetricData('sallyKung', "sleep")
sallySleep.tail(5)

,start time,start time ISO,start time timezone,start time offset,end time,end time ISO,end time timezone,end time offset,light mins,deep mins,...,toss turns,type,actual seconds,calories,heart rate avg,heart rate min,heart rate max,state,version,id
0,2016-05-05 06:31:20,2016-05-05T06:31:20Z,America/New_York,-240,2016-05-05 11:03:20,2016-05-05T11:03:20Z,America/New_York,-240,157,34,...,18,sleep,16320,311.600006,65.787804,NaN,NaN,complete,2,sleep__2016-05-05T063120
0,2016-05-06 04:28:25,2016-05-06T04:28:25Z,America/New_York,-240,2016-05-06 14:10:23,2016-05-06T14:10:23Z,America/New_York,-240,276,132,...,41,sleep,33900,590.564026,64.997345,NaN,NaN,complete,2,sleep__2016-05-06T042825
0,2016-05-07 05:22:32,2016-05-07T05:22:32Z,America/New_York,-240,2016-05-07 17:12:00,2016-05-07T17:12:00Z,America/New_York,-240,366,145,...,40,sleep,42540,812.099976,63.937489,NaN,NaN,complete,2,sleep__2016-05-07T052232
0,2016-05-08 07:50:37,2016-05-08T07:50:37Z,America/New_York,-240,2016-05-08 15:52:56,2016-05-08T15:52:56Z,America/New_York,-240,342,30,...,35,sleep,28200,476.764008,61.165691,NaN,NaN,complete,2,sleep__2016-05-08T075037
0,2016-05-09 04:51:23,2016-05-09T04:51:23Z,America/New_York,-240,2016-05-09 12:03:33,2016-05-09T12:03:33Z,America/New_York,-240,195,106,...,31,sleep,25920,494.600006,60.591930,NaN,NaN,complete,2,sleep__2016-05-09T045123


In [10]:
sallySleep.dtypes

start time              object
start time ISO          object
start time timezone     object
start time offset      float64
end time                object
end time ISO            object
end time timezone       object
end time offset        float64
light mins             float64
deep mins              float64
rem mins               float64
interruption mins      float64
unknown mins           float64
interruptions          float64
toss turns             float64
type                    object
actual seconds         float64
calories               float64
heart rate avg         float64
heart rate min         float64
heart rate max         float64
state                   object
version                float64
id                      object
dtype: object

In [11]:
sallyJournal = loadMetricData('sallyKung', "journal")
sallyJournal.tail(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
41,5/6/16 12:49,5/6/16 13:14,5/6/16,12:49 PM,1:14 PM,Orals,"79 5th Ave, 16th floor",Anxious,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,5/6/16 13:14,5/6/16 13:30,5/6/16,1:14 PM,1:30 PM,Talking to Abby,"79 5th Ave, 16th floor",Calm,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,5/6/16 13:47,5/6/16 15:00,5/6/16,1:47 PM,3:00 PM,With Sarah and Hashem,Durst,Angry,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,5/6/16 15:10,5/6/16 15:30,5/6/16,3:10 PM,3:30 PM,Eating fish and chips with Sarah,"66 12th Street, 407",Happy,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,5/6/16 16:21,5/6/16 16:21,5/6/16,4:21 PM,4:21 PM,Sarah messages me on Google,"66 12th Street, 407",Annoyed,1,NaN,NaN,NaN,NaN,1,NaN,1,NaN,1


In [12]:
sallyJournal.dtypes

starttime       object
endtime         object
date            object
start           object
end             object
activity        object
location        object
generalemo      object
qualitative      int64
excited        float64
happy          float64
calm           float64
anxious        float64
sad            float64
angry          float64
hungry         float64
tired          float64
bored          float64
dtype: object

In [13]:
# data= barbJournal.iloc[:0]
# clean metric data
sallyJournal["starttime"]= pd.to_datetime(sallyJournal.starttime)

# data= barbJournal.iloc[:1]
sallyJournal["endtime"]= pd.to_datetime(sallyJournal.endtime)


# sarahJournal.qualitative = sarahJournal.qualitative.convert_objects(convert_numeric=True)
# sarahJournal.excited = sarahJournal.excited.convert_objects(convert_numeric=True)
# sarahJournal.happy = sarahJournal.happy.convert_objects(convert_numeric=True)

sallyJournal.qualitative = sallyJournal.qualitative.fillna(0)
sallyJournal.excited = sallyJournal.excited.fillna(0)
sallyJournal.happy = sallyJournal.happy.fillna(0)
sallyJournal.calm = sallyJournal.calm.fillna(0)
sallyJournal.anxious = sallyJournal.anxious.fillna(0)
sallyJournal.sad = sallyJournal.sad.fillna(0)
sallyJournal.angry = sallyJournal.angry.fillna(0)
sallyJournal.hungry = sallyJournal.hungry.fillna(0)
sallyJournal.tired = sallyJournal.tired.fillna(0)
sallyJournal.bored = sallyJournal.bored.fillna(0)

print(sallyJournal.shape)
sallyJournal.head(5)
# sallyJournal.tail(5)

(46, 18)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,happy,calm,anxious,sad,angry,hungry,tired,bored
0,2016-05-02 13:25:00,2016-05-02 13:28:00,5/2/16,1:25 PM,1:28 PM,Trying on watch,Barb's Office,Excited,8,8,8,0,0,0,0,0,0,0
1,2016-05-02 13:40:00,2016-05-02 00:00:00,5/2/16,1:40 PM,NaN,Editing photos for work,Barb's Office,Streeeeeessed,6,0,0,0,0,0,0,0,0,0
2,2016-05-02 13:48:00,2016-05-02 13:48:00,5/2/16,1:48 PM,1:48 PM,Wafer fit perfectly on top of cup,Barb's Office,HAPPY,10,10,10,0,0,0,0,0,0,0
3,2016-05-02 14:31:00,2016-05-02 14:50:00,5/2/16,2:31 PM,2:50 PM,Sorting through magnetic pins,Barb's Office,Anxious,5,0,0,0,5,0,6,0,0,0
4,2016-05-02 15:10:00,2016-05-02 15:14:00,5/2/16,3:10 PM,3:14 PM,Trying to figure out where photos went,Barb's Office,"ANXIOUS, CONFUSED",9,0,0,0,9,0,0,0,0,0


In [14]:

sallyJournal['boolexcited'] =  sallyJournal['excited'] > 0
sallyJournal['boolhappy'] =  sallyJournal['happy'] > 0
sallyJournal['boolcalm'] =  sallyJournal['calm'] > 0
sallyJournal['boolanxious'] =  sallyJournal['anxious'] > 0
sallyJournal['boolsad'] =  sallyJournal['sad'] > 0
sallyJournal['boolangry'] =  sallyJournal['angry'] > 0
sallyJournal['booltired'] =  sallyJournal['tired'] > 0
sallyJournal['boolhungry'] =  sallyJournal['hungry'] > 0
sallyJournal['boolbored'] =  sallyJournal['bored'] > 0


for i in range(0, sallyJournal.shape[0]):
#     sallyJournal['intqual'] = int(sallyJournal.iloc[i, 8])
    sallyJournal['intexcited'] = int(sallyJournal.iloc[i, 9])
    sallyJournal['inthappy'] = int(sallyJournal.iloc[i, 10]) 
    sallyJournal['intcalm'] = int(sallyJournal.iloc[i, 11])
    sallyJournal['intanxious'] = int(sallyJournal.iloc[i, 12])
    sallyJournal['intsad'] = int(sallyJournal.iloc[i, 13]) 
    sallyJournal['intangry'] = int(sallyJournal.iloc[i, 14]) 
    sallyJournal['inttired'] = int(sallyJournal.iloc[i, 15]) 
    sallyJournal['inthungry'] = int(sallyJournal.iloc[i, 16]) 
    sallyJournal['intbored'] = int(sallyJournal.iloc[i, 17]) 
    
    
sallyJournal['exercise'] = sallyJournal['activity'].str.contains('exercise')
sallyJournal['thesis'] = sallyJournal['activity'].str.contains('thesis')
sallyJournal['sarah'] = sallyJournal['activity'].str.contains('sarah')
sallyJournal['barb'] = sallyJournal['activity'].str.contains('barb')
sallyJournal['sol'] = sallyJournal['activity'].str.contains('sol')
sallyJournal['arash'] = sallyJournal['activity'].str.contains('arash')

sallyJournal['home'] = sallyJournal['location'].str.contains('home')
sallyJournal['work'] = sallyJournal['location'].str.contains("barb's office")







boolexcited = sallyJournal.iloc[:, 18].values.reshape(sallyJournal.shape[0], 1)
boolhappy = sallyJournal.iloc[:, 19].values.reshape(sallyJournal.shape[0], 1)
boolcalm = sallyJournal.iloc[:, 20].values.reshape(sallyJournal.shape[0], 1)
boolanxious = sallyJournal.iloc[:, 21].values.reshape(sallyJournal.shape[0], 1)
boolsad = sallyJournal.iloc[:, 22].values.reshape(sallyJournal.shape[0], 1)
boolangry = sallyJournal.iloc[:, 23].values.reshape(sallyJournal.shape[0], 1)
booltired = sallyJournal.iloc[:, 24].values.reshape(sallyJournal.shape[0], 1)
boolhungry = sallyJournal.iloc[:, 25].values.reshape(sallyJournal.shape[0], 1)
boolbored = sallyJournal.iloc[:, 26].values.reshape(sallyJournal.shape[0], 1)
intexcited = sallyJournal.iloc[:, 27].values.reshape(sallyJournal.shape[0], 1)
inthappy = sallyJournal.iloc[:, 28].values.reshape(sallyJournal.shape[0], 1)
intcalm = sallyJournal.iloc[:, 29].values.reshape(sallyJournal.shape[0], 1)
intanxious = sallyJournal.iloc[:, 30].values.reshape(sallyJournal.shape[0], 1)
intsad = sallyJournal.iloc[:, 31].values.reshape(sallyJournal.shape[0], 1)
intangry = sallyJournal.iloc[:, 32].values.reshape(sallyJournal.shape[0], 1)
inttired = sallyJournal.iloc[:, 33].values.reshape(sallyJournal.shape[0], 1)
inthungry = sallyJournal.iloc[:, 34].values.reshape(sallyJournal.shape[0], 1)
intbored = sallyJournal.iloc[:, 35].values.reshape(sallyJournal.shape[0], 1)
exercise = sallyJournal.iloc[:, 36].values.reshape(sallyJournal.shape[0], 1)
thesis = sallyJournal.iloc[:, 37].values.reshape(sallyJournal.shape[0], 1)
sarah = sallyJournal.iloc[:, 38].values.reshape(sallyJournal.shape[0], 1)
barb = sallyJournal.iloc[:, 39].values.reshape(sallyJournal.shape[0], 1)
sol = sallyJournal.iloc[:, 40].values.reshape(sallyJournal.shape[0], 1)
arash = sallyJournal.iloc[:, 41].values.reshape(sallyJournal.shape[0], 1)
work = sallyJournal.iloc[:, 42].values.reshape(sallyJournal.shape[0], 1)
home = sallyJournal.iloc[:, 43].values.reshape(sallyJournal.shape[0], 1)

sallyJournal.head(5)

,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,inthungry,intbored,exercise,thesis,sarah,barb,sol,arash,home,work
0,2016-05-02 13:25:00,2016-05-02 13:28:00,5/2/16,1:25 PM,1:28 PM,Trying on watch,Barb's Office,Excited,8,8,...,0,1,False,False,False,False,False,False,False,False
1,2016-05-02 13:40:00,2016-05-02 00:00:00,5/2/16,1:40 PM,NaN,Editing photos for work,Barb's Office,Streeeeeessed,6,0,...,0,1,False,False,False,False,False,False,False,False
2,2016-05-02 13:48:00,2016-05-02 13:48:00,5/2/16,1:48 PM,1:48 PM,Wafer fit perfectly on top of cup,Barb's Office,HAPPY,10,10,...,0,1,False,False,False,False,False,False,False,False
3,2016-05-02 14:31:00,2016-05-02 14:50:00,5/2/16,2:31 PM,2:50 PM,Sorting through magnetic pins,Barb's Office,Anxious,5,0,...,0,1,False,False,False,False,False,False,False,False
4,2016-05-02 15:10:00,2016-05-02 15:14:00,5/2/16,3:10 PM,3:14 PM,Trying to figure out where photos went,Barb's Office,"ANXIOUS, CONFUSED",9,0,...,0,1,False,False,False,False,False,False,False,False


In [15]:
# barbJournal.dtypes

In [16]:
# barbJournal.excited.convert_objects(convert_numeric=True)

In [17]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [18]:
# indicator for walking
totalData = makeIndicators('excitement', sallyMetrics, sallyJournal, 0, 1, 18)
totalData = makeIndicators('happy', sallyMetrics, sallyJournal, 0, 1, 19)
totalData = makeIndicators('calm', sallyMetrics, sallyJournal, 0, 1, 20)
totalData = makeIndicators('anxious', sallyMetrics, sallyJournal, 0, 1, 21)
totalData = makeIndicators('sad', sallyMetrics, sallyJournal, 0, 1, 22)
totalData = makeIndicators('angry', sallyMetrics, sallyJournal, 0, 1, 23)
totalData = makeIndicators('hungry', sallyMetrics, sallyJournal, 0, 1, 24)
totalData = makeIndicators('tired', sallyMetrics, sallyJournal, 0, 1, 25)
totalData = makeIndicators('bored', sallyMetrics, sallyJournal, 0, 1, 26)

totalData = makeIndicators('exercise', sallyMetrics, sallyJournal, 0, 1, 36)
totalData = makeIndicators('thesis', sallyMetrics, sallyJournal, 0, 1, 37)
totalData = makeIndicators('sarah', sallyMetrics, sallyJournal, 0, 1, 38)
totalData = makeIndicators('barb', sallyMetrics, sallyJournal, 0, 1, 39)
totalData = makeIndicators('sol', sallyMetrics, sallyJournal, 0, 1, 40)
totalData = makeIndicators('arash', sallyMetrics, sallyJournal, 0, 1, 41)
totalData = makeIndicators('work', sallyMetrics, sallyJournal, 0, 1, 42)
totalData = makeIndicators('home', sallyMetrics, sallyJournal, 0, 1, 43)

In [19]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  11520.000000  11520.000000  11520.000000  11520.000000  11520.000000   
mean      69.752778      3.829514      1.741179      0.001160     74.834089   
std       30.776918     17.035531      1.311452      0.047230     29.865726   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%       62.000000      0.000000      1.200000      0.000052     81.500000   
50%       76.000000      0.000000      1.300000      0.000055     86.000000   
75%       88.000000      0.000000      1.800000      0.000060     88.700000   
max      156.000000    142.000000     11.400000      4.270000     99.500000   

            airtemp   excitement       happy       calm    anxious  ...    \
count  11520.000000        11520       11520      11520      11520  ...     
mean      72.381589  0.000434028  0.00190972  0.0237847  0.0703125  ...     
std       28.967882    0.0208297   0.0436605   0.152384  

In [20]:
print(totalData.head())

            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-05-02 00:00:00          0      4     1.709    0         0        0   
1 2016-05-02 00:01:00          0      4     1.709    0         0        0   
2 2016-05-02 00:02:00          0      4     1.709    0         0        0   
3 2016-05-02 00:03:00          0      4     1.709    0         0        0   
4 2016-05-02 00:04:00          0      4     1.709    0         0        0   

  excitement  happy   calm  ...    tired  bored exercise thesis  sarah   barb  \
0      False  False  False  ...    False  False    False  False  False  False   
1      False  False  False  ...    False  False    False  False  False  False   
2      False  False  False  ...    False  False    False  False  False  False   
3      False  False  False  ...    False  False    False  False  False  False   
4      False  False  False  ...    False  False    False  False  False  False   

     sol  arash   work   home  
0  False  False  F

In [21]:
totalData.to_csv('sallyTotal.csv')

In [22]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,tired,bored,exercise,thesis,sarah,barb,sol,arash,work,home
0,2016-05-02 00:00:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2016-05-02 00:01:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2016-05-02 00:02:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2016-05-02 00:03:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2016-05-02 00:04:00,0,4,1.709,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(11520, 28)
            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-05-02 00:00:00          0      4     1.709    0         0        0   
1 2016-05-02 00:01:00          0      4     1.709    0         0        0   
2 2016-05-02 00:02:00          0      4     1.709    0         0        0   
3 2016-05-02 00:03:00          0      4     1.709    0         0        0   
4 2016-05-02 00:04:00          0      4     1.709    0         0        0   

  excitement  happy   calm ...  sarah   barb    sol  arash   work   home  \
0      False  False  False ...  False  False  False  False  False  False   
1      False  False  False ...  False  False  False  False  False  False   
2      False  False  False ...  False  False  False  False  False  False   
3      False  False  False ...  False  False  False  False  False  False   
4      False  False  False ...  False  False  False  False  False  False   

  weekday month hour min  
0       0     5    0   0  
1       0     

In [24]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
thesis = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
sarah = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
barb = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
sol = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
arash = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
work = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
home = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 27].values.reshape(totalData.shape[0], 1)

In [25]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
#         exercise, 
#         thesis, 
#         sarah, 
#         barb, 
#         sol, 
#         arash, 
#         work, 
#         home, 
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(11520, 9)

In [26]:
# define y
y = totalData.excitement.values
y.shape

(11520,)

In [27]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [28]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [29]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

# Excited Model Results

In [30]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000496031746032 0.000289351851852
 Accuracy Rate    0.999503968254    0.999710648148
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00028935185185185184, 0.99971064814814814, 0.0, 1.0)

In [31]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set       Test Set    
------------------- ----------------- ----------------
         Error Rate 0.000496031746032 0.00115740740741
      Accuracy Rate    0.999503968254   0.998842592593
     True Positives               0.0              0.0
     True Negatives                --   0.999131693198


(0.0011574074074074073, 0.99884259259259256, 0.0, 0.99913169319826334)

In [32]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00111607142857 0.000868055555556
 Accuracy Rate   0.998883928571    0.999131944444
True Positives              0.0               0.0
True Negatives               --    0.999421128799


(0.00086805555555555551, 0.99913194444444442, 0.0, 0.99942112879884226)

In [33]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000496031746032 0.000289351851852
 Accuracy Rate    0.999503968254    0.999710648148
True Positives               0.0               0.0
True Negatives                --               1.0


(0.00028935185185185184, 0.99971064814814814, 0.0, 1.0)

In [34]:
# define y
y = totalData.happy.values
y.shape

(11520,)

In [35]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [36]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Happy Model Results

In [37]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00210813492063 0.00144675925926
 Accuracy Rate   0.997891865079   0.998553240741
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0014467592592592592, 0.9985532407407407, 0.0, 1.0)

In [38]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00210813492063 0.00144675925926
      Accuracy Rate   0.997891865079   0.998553240741
     True Positives              0.0              0.0
     True Negatives               --              1.0


(0.0014467592592592592, 0.9985532407407407, 0.0, 1.0)

In [39]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0128968253968 0.0127314814815
 Accuracy Rate  0.987103174603  0.987268518519
True Positives 0.0588235294118             0.0
True Negatives              --  0.988698927847


(0.012731481481481481, 0.98726851851851849, 0.0, 0.98869892784700086)

In [40]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0223214285714 0.0248842592593
 Accuracy Rate  0.977678571429  0.975115740741
True Positives 0.0588235294118             0.0
True Negatives              --  0.976528542451


(0.024884259259259259, 0.9751157407407407, 0.0, 0.97652854245146337)

In [41]:
# define y
y = totalData.calm.values
y.shape

(11520,)

In [42]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [43]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Calm Model Results

In [44]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0224454365079 0.0271990740741
 Accuracy Rate  0.977554563492  0.972800925926
True Positives             0.0             0.0
True Negatives              --             1.0


(0.027199074074074073, 0.97280092592592593, 0.0, 1.0)

In [45]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0233134920635 0.0280671296296
      Accuracy Rate  0.976686507937   0.97193287037
     True Positives             0.0             0.0
     True Negatives              --  0.999107674004


(0.028067129629629629, 0.97193287037037035, 0.0, 0.99910767400356926)

In [46]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0411706349206  0.044849537037
 Accuracy Rate  0.958829365079  0.955150462963
True Positives 0.0666666666667 0.0531914893617
True Negatives              --  0.980368828079


(0.044849537037037035,
 0.95515046296296302,
 0.053191489361702128,
 0.98036882807852466)

In [47]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0481150793651 0.0549768518519
 Accuracy Rate  0.951884920635  0.945023148148
True Positives  0.116666666667 0.0957446808511
True Negatives              --  0.968768590125


(0.054976851851851853,
 0.94502314814814814,
 0.095744680851063829,
 0.96876859012492567)

In [48]:
# define y
y = totalData.anxious.values
y.shape

(11520,)

In [49]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [50]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Anxious Model Results

In [51]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set     Test Set   
-------------- --------------- --------------
    Error Rate 0.0951140873016 0.100115740741
 Accuracy Rate  0.904885912698 0.899884259259
True Positives  0.228070175439          0.175
True Negatives              -- 0.953980099502


(0.10011574074074074,
 0.8998842592592593,
 0.17499999999999999,
 0.95398009950248752)

In [52]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- ---------------- --------------
         Error Rate  0.0721726190476      0.0703125
      Accuracy Rate   0.927827380952      0.9296875
     True Positives 0.00175438596491            0.0
     True Negatives               -- 0.999067164179


(0.0703125, 0.9296875, 0.0, 0.99906716417910446)

In [53]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0706845238095 0.0694444444444
 Accuracy Rate   0.92931547619  0.930555555556
True Positives             0.0             0.0
True Negatives              --             1.0


(0.069444444444444448, 0.93055555555555558, 0.0, 1.0)

In [54]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set    
-------------- --------------- ----------------
    Error Rate 0.0730406746032  0.0726273148148
 Accuracy Rate  0.926959325397   0.927372685185
True Positives 0.0228070175439 0.00833333333333
True Negatives              --   0.995957711443


(0.072627314814814811,
 0.92737268518518523,
 0.0083333333333333332,
 0.99595771144278611)

In [55]:
# define y
y = totalData.sad.values
y.shape

(11520,)

In [56]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [57]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Sad Model Values

In [58]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [59]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set   Test Set
------------------- ----------------- --------
         Error Rate 0.000124007936508      0.0
      Accuracy Rate    0.999875992063      1.0
     True Positives               0.0      nan
     True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [60]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00124007936508 0.000578703703704
 Accuracy Rate   0.998759920635    0.999421296296
True Positives              0.0               nan
True Negatives               --    0.999421296296


(0.00057870370370370367, 0.99942129629629628, nan, 0.99942129629629628)

In [61]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [62]:
# define y
y = totalData.angry.values
y.shape

(11520,)

In [63]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [64]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Angry Model Results

In [65]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00508432539683 0.00376157407407
 Accuracy Rate   0.994915674603   0.996238425926
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0037615740740740739, 0.99623842592592593, 0.0, 1.0)

In [66]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set       Test Set    
------------------- ---------------- ----------------
         Error Rate 0.00520833333333 0.00376157407407
      Accuracy Rate   0.994791666667   0.996238425926
     True Positives              0.0              0.0
     True Negatives               --              1.0


(0.0037615740740740739, 0.99623842592592593, 0.0, 1.0)

In [67]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00508432539683 0.00376157407407
 Accuracy Rate   0.994915674603   0.996238425926
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0037615740740740739, 0.99623842592592593, 0.0, 1.0)

In [68]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00508432539683 0.00376157407407
 Accuracy Rate   0.994915674603   0.996238425926
True Positives              0.0              0.0
True Negatives               --              1.0


(0.0037615740740740739, 0.99623842592592593, 0.0, 1.0)

In [69]:
# define y
y = totalData.hungry.values
y.shape

(11520,)

In [70]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [71]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Hungry Model Results

In [72]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.497395833333 0.510127314815
 Accuracy Rate 0.502604166667 0.489872685185
True Positives 0.824953445065 0.823008849558
True Negatives             -- 0.466563467492


(0.51012731481481477,
 0.48987268518518523,
 0.82300884955752207,
 0.46656346749226008)

In [73]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0780009920635 0.0763888888889
      Accuracy Rate  0.921999007937  0.923611111111
     True Positives  0.143389199255  0.110619469027
     True Negatives              --  0.980495356037


(0.076388888888888895,
 0.92361111111111116,
 0.11061946902654868,
 0.98049535603715166)

In [74]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate  0.0664682539683 0.0656828703704
 Accuracy Rate   0.933531746032   0.93431712963
True Positives 0.00186219739292             0.0
True Negatives               --  0.999690402477


(0.065682870370370364, 0.93431712962962965, 0.0, 0.9996904024767802)

In [75]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0678323412698 0.0656828703704
 Accuracy Rate   0.93216765873   0.93431712963
True Positives             0.0             0.0
True Negatives              --  0.999690402477


(0.065682870370370364, 0.93431712962962965, 0.0, 0.9996904024767802)

In [76]:
# define y
y = totalData.tired.values
y.shape

(11520,)

In [77]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [78]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Tired Model Results

In [79]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [80]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set   Test Set
------------------- ----------------- --------
         Error Rate 0.000124007936508      0.0
      Accuracy Rate    0.999875992063      1.0
     True Positives               0.0      nan
     True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [81]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00124007936508 0.000578703703704
 Accuracy Rate   0.998759920635    0.999421296296
True Positives              0.0               nan
True Negatives               --    0.999421296296


(0.00057870370370370367, 0.99942129629629628, nan, 0.99942129629629628)

In [82]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [83]:
# define y
y = totalData.bored.values
y.shape

(11520,)

In [84]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [85]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Bored Model Results

In [86]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [87]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set   Test Set
------------------- ----------------- --------
         Error Rate 0.000124007936508      0.0
      Accuracy Rate    0.999875992063      1.0
     True Positives               0.0      nan
     True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)

In [88]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00124007936508 0.000578703703704
 Accuracy Rate   0.998759920635    0.999421296296
True Positives              0.0               nan
True Negatives               --    0.999421296296


(0.00057870370370370367, 0.99942129629629628, nan, 0.99942129629629628)

In [89]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set   Test Set
-------------- ----------------- --------
    Error Rate 0.000124007936508      0.0
 Accuracy Rate    0.999875992063      1.0
True Positives               0.0      nan
True Negatives                --      1.0


(0.0, 1.0, nan, 1.0)